# Greedy Scheduler

Used for testing to ensure that the Factory Env Works

In [3]:
import json
from pathlib import Path

project_root = Path().resolve().parent.parent
data_file = project_root / "data" / "Input_JSON_Schedule_Optimization.json"
with open(data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)
# data

In [4]:
from utils.input_schemas import ProductRequest

# product_requests = [
#     ProductRequest(product='WOVEN LABEL OURELA FABRIC', amount=25, deadline=None),
#     ProductRequest(product='ESTORE RIBBON', amount=10, deadline=None),
#     ProductRequest(product='GORGORÃO RIBBON', amount=1, deadline=None),
#     ProductRequest(product='TAFFETA EMBLEM', amount=1, deadline=None),
#     ProductRequest(product='PRINTED RIBBON', amount=1, deadline=None),
#     ProductRequest(product='ROK ELASTIC', amount=3, deadline=None),
#     ProductRequest(product='BRAIDED ELASTIC', amount=1, deadline=None),
#     ProductRequest(product='SMOOTH ELASTIC(A)', amount=1, deadline=None),
#     ProductRequest(product='STRAP W/ STRIPES', amount=0, deadline=None),
# ]

# use input data from json file
product_requests_data = data["product_requests"]
product_requests = [ProductRequest(**pr) for pr in product_requests_data]
product_requests

[ProductRequest(product='WOVEN LABEL OURELA FABRIC', amount=13, deadline=None),
 ProductRequest(product='ESTORE RIBBON', amount=6, deadline=None),
 ProductRequest(product='WOVEN LABEL OURELA CUT', amount=5, deadline=None),
 ProductRequest(product='GORGORÃO RIBBON', amount=13, deadline=None),
 ProductRequest(product='GORGORÃO RIBBON W/ INSCR', amount=5, deadline=None),
 ProductRequest(product='TAFFETA EMBLEM', amount=10, deadline=None),
 ProductRequest(product='PRINTED RIBBON', amount=30, deadline=None),
 ProductRequest(product='ELASTIC W/ INSCR', amount=2, deadline=None),
 ProductRequest(product='ELASTIC W/ INSCR', amount=1, deadline=960),
 ProductRequest(product='ROK ELASTIC', amount=3, deadline=None),
 ProductRequest(product='SMOOTH STRAP(A)', amount=4, deadline=None),
 ProductRequest(product='BRAIDED ELASTIC', amount=15, deadline=None),
 ProductRequest(product='PRINTED RIBBON  PES', amount=10, deadline=None),
 ProductRequest(product='SMOOTH ELASTIC(A)', amount=5, deadline=None),
 Pr

In [6]:
from utils.input_schemas import EnergySource
from utils.job_builder import JobBuilder
from utils.factory_logic_loader import FactoryLogicLoader

steps = 1152
energy_sources = [EnergySource(**es) for es in data["energy_sources"]]

factory_logic = FactoryLogicLoader.load_from_file(filepath=str(data_file))
job_builder = JobBuilder(factory_logic=factory_logic)
jobs = job_builder.build_jobs(product_requests=product_requests)

total_operations = sum(len(job.operations) for job in jobs)
print(f"Total operations: {total_operations}")

Total operations: 275


In [7]:
from factory.factory import Factory
from schedulers.greedy.greedy_scheduler import GreedyScheduler

factory = Factory(factory_logic=factory_logic)
factory.add_jobs(jobs=jobs)
state = factory.get_factory_state()

scheduler = GreedyScheduler(greedy_type="min_power")

total_power_cost = 0
while not factory.done() and factory.current_step < steps:
    chosen_actions = scheduler.choose(state)
    factory.apply_actions(chosen_actions)
    step_info = factory.step()
    total_power_cost += step_info['step_power_cost']
    state = factory.get_factory_state()


scheduled_actions = scheduler.scheduled_actions
scheduled_actions = [
    action for action in scheduled_actions
    if action is not None
]

print(f"Energy Cost Total: {total_power_cost}")
print(f"Number of jobs not done: {len([job for job in factory.jobs if not job.done])}")
print(f"Number of jobs done: {len([job for job in factory.jobs if job.done])}")

scheduled_actions = scheduler.scheduled_actions
# remove None objects
scheduled_actions = [
    action for action in scheduled_actions
    if action is not None
]

scheduler.plot_gantt_by_product(factory, scheduled_actions)

ModuleNotFoundError: No module named 'factory.factory_schemas'